In [1]:
# Load the data and get a quick sense 
path = "wasb:///example/data/Incidents.csv"
data=sc.textFile(path)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1515736974360_0004,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
data.take(10)

[u'OBJECTID,Identifier,Occurrence Date,Day of Week,Occurrence Month,Occurrence Day,Occurrence Year,Occurrence Hour,CompStat Month,CompStat Day,CompStat Year,Offense,Offense Classification,Sector,Precinct,Borough,Jurisdiction,XCoordinate,YCoordinate,Location 1', u'1,f070032d,09/06/1940 07:30:00 PM,Friday,Sep,6,1940,19,9,7,2010,BURGLARY,FELONY,D,66,BROOKLYN,N.Y. POLICE DEPT,987478,166141,"(40.6227027620001, -73.9883732929999)"', u'2,c6245d4d,12/14/1968 12:20:00 AM,Saturday,Dec,14,1968,0,12,14,2008,GRAND LARCENY,FELONY,G,28,MANHATTAN,N.Y. POLICE DEPT,996470,232106,"(40.8037530600001, -73.955861904)"', u'3,716dbc6f,10/30/1970 03:30:00 PM,Friday,Oct,30,1970,15,10,31,2008,BURGLARY,FELONY,H,84,BROOKLYN,N.Y. POLICE DEPT,986508,190249,"(40.688874254, -73.9918594329999)"', u'4,638cd7b7,07/18/1972 11:00:00 PM,Tuesday,Jul,18,1972,23,7,19,2012,GRAND LARCENY OF MOTOR VEHICLE,FELONY,F,73,BROOKLYN,N.Y. POLICE DEPT,1005876,182440,"(40.6674141890001, -73.9220463899999)"', u'5,6e410287,05/21/1987 12:01:0

In [3]:
# Filter the header row 
header = data.first()
print header

OBJECTID,Identifier,Occurrence Date,Day of Week,Occurrence Month,Occurrence Day,Occurrence Year,Occurrence Hour,CompStat Month,CompStat Day,CompStat Year,Offense,Offense Classification,Sector,Precinct,Borough,Jurisdiction,XCoordinate,YCoordinate,Location 1

In [4]:
dataWoHeader = data.filter(lambda x: x<>header)

In [5]:
dataWoHeader.first()

u'1,f070032d,09/06/1940 07:30:00 PM,Friday,Sep,6,1940,19,9,7,2010,BURGLARY,FELONY,D,66,BROOKLYN,N.Y. POLICE DEPT,987478,166141,"(40.6227027620001, -73.9883732929999)"'

In [6]:
# Parse the rows to extract fields 
dataWoHeader.map(lambda x:x.split(",")).take(10)

[[u'1', u'f070032d', u'09/06/1940 07:30:00 PM', u'Friday', u'Sep', u'6', u'1940', u'19', u'9', u'7', u'2010', u'BURGLARY', u'FELONY', u'D', u'66', u'BROOKLYN', u'N.Y. POLICE DEPT', u'987478', u'166141', u'"(40.6227027620001', u' -73.9883732929999)"'], [u'2', u'c6245d4d', u'12/14/1968 12:20:00 AM', u'Saturday', u'Dec', u'14', u'1968', u'0', u'12', u'14', u'2008', u'GRAND LARCENY', u'FELONY', u'G', u'28', u'MANHATTAN', u'N.Y. POLICE DEPT', u'996470', u'232106', u'"(40.8037530600001', u' -73.955861904)"'], [u'3', u'716dbc6f', u'10/30/1970 03:30:00 PM', u'Friday', u'Oct', u'30', u'1970', u'15', u'10', u'31', u'2008', u'BURGLARY', u'FELONY', u'H', u'84', u'BROOKLYN', u'N.Y. POLICE DEPT', u'986508', u'190249', u'"(40.688874254', u' -73.9918594329999)"'], [u'4', u'638cd7b7', u'07/18/1972 11:00:00 PM', u'Tuesday', u'Jul', u'18', u'1972', u'23', u'7', u'19', u'2012', u'GRAND LARCENY OF MOTOR VEHICLE', u'FELONY', u'F', u'73', u'BROOKLYN', u'N.Y. POLICE DEPT', u'1005876', u'182440', u'"(40.667414

In [7]:
import csv
from StringIO import StringIO
from collections import namedtuple


In [9]:
fields   = header.replace(" ","_").replace("/","_").split(",")
print(fields)

[u'OBJECTID', u'Identifier', u'Occurrence_Date', u'Day_of_Week', u'Occurrence_Month', u'Occurrence_Day', u'Occurrence_Year', u'Occurrence_Hour', u'CompStat_Month', u'CompStat_Day', u'CompStat_Year', u'Offense', u'Offense_Classification', u'Sector', u'Precinct', u'Borough', u'Jurisdiction', u'XCoordinate', u'YCoordinate', u'Location_1']

In [11]:
Crime   = namedtuple('Crime', fields)

In [12]:
def parse(row):
    reader = csv.reader(StringIO(row))
    row=reader.next()
    return Crime(*row)

In [13]:
crimes=dataWoHeader.map(parse)

In [14]:
crimes.first()

Crime(OBJECTID='1', Identifier='f070032d', Occurrence_Date='09/06/1940 07:30:00 PM', Day_of_Week='Friday', Occurrence_Month='Sep', Occurrence_Day='6', Occurrence_Year='1940', Occurrence_Hour='19', CompStat_Month='9', CompStat_Day='7', CompStat_Year='2010', Offense='BURGLARY', Offense_Classification='FELONY', Sector='D', Precinct='66', Borough='BROOKLYN', Jurisdiction='N.Y. POLICE DEPT', XCoordinate='987478', YCoordinate='166141', Location_1='(40.6227027620001, -73.9883732929999)')

In [15]:
crimes.first().Offense

'BURGLARY'

In [16]:
crimes.map(lambda x:x.Offense).countByValue()

defaultdict(<type 'int'>, {'FELONY ASSAULT': 184042, 'MURDER & NON-NEGL. MANSLAUGHTE': 4574, 'NA': 1, 'GRAND LARCENY OF MOTOR VEHICLE': 101963, 'ROBBERY': 198744, 'BURGLARY': 191369, 'GRAND LARCENY': 428993, 'RAPE': 13779})

In [17]:
crimes.map(lambda x:x.Occurrence_Year).countByValue()

defaultdict(<type 'int'>, {'': 244, '1958': 1, '1974': 3, '1960': 1, '1976': 2, '1920': 1, '1908': 1, '1910': 3, '1945': 2, '1977': 3, '1986': 2, '1987': 6, '1984': 4, '1985': 8, '1968': 1, '1983': 1, '1980': 5, '1981': 1, '1964': 1, '1965': 2, '1940': 1, '1946': 1, '1988': 6, '1989': 12, '2015': 102657, '2014': 106849, '2011': 107206, '2010': 105643, '2013': 111286, '2012': 111798, '1979': 6, '1978': 2, '1955': 1, '1954': 1, '1956': 1, '1905': 2, '1911': 1, '1982': 5, '1913': 4, '1912': 1, '1915': 3, '1914': 2, '1969': 1, '1991': 12, '1990': 17, '1993': 23, '1992': 12, '1995': 27, '1994': 19, '1997': 40, '1996': 34, '1999': 124, '1998': 74, '1975': 2, '1959': 1, '1973': 5, '1972': 2, '1971': 1, '1970': 2, '2002': 368, '2003': 490, '2000': 282, '2001': 343, '2006': 127887, '2007': 120554, '2004': 692, '2005': 3272, '2008': 117375, '2009': 106018, '1966': 7, '1950': 1})

In [18]:
crimesFiltered=crimes.filter(lambda x : not (x.Offense=="NA" or x.Occurrence_Year==''))\
                .filter(lambda x: int(x.Occurrence_Year)>=2006)

In [19]:
crimesFiltered.map(lambda x:x.Occurrence_Year).countByValue()

defaultdict(<type 'int'>, {'2006': 127887, '2007': 120554, '2015': 102657, '2014': 106849, '2008': 117375, '2009': 106018, '2011': 107206, '2010': 105643, '2013': 111286, '2012': 111798})

In [20]:
def extractCoords(location):
    location_lat = float(location[1:location.index(",")])
    location_lon = float(location[location.index(",")+1:-1])
    return (location_lat,location_lon)

In [21]:
crimesFiltered.map(lambda x:extractCoords(x.Location_1))\
            .reduce(lambda x,y:(min(x[0],y[0]),min(x[1],y[1]))) # Find the minimum latitude and longitute in the data set

(40.112709974, -77.519206334)

In [22]:
crimesFiltered.map(lambda x:extractCoords(x.Location_1))\
            .reduce(lambda x,y:(max(x[0],y[0]),max(x[1],y[1]))) # find the max lat & long in the data set. These seem to be falling outside NY space

(59.5805088160001, -73.700716685)

In [23]:
crimesFinal =  crimesFiltered.filter(lambda x: extractCoords(x.Location_1)[0]>=40.477399 and \
                                    extractCoords(x.Location_1)[0]<=40.917577 and \
                                    extractCoords(x.Location_1)[1]>=-74.25909 and  \
                                    extractCoords(x.Location_1)[1]<=-73.700009)

In [24]:
# Trend by Year
crimesFinal.map(lambda x:x.Occurrence_Year).countByValue()

defaultdict(<type 'int'>, {'2006': 127887, '2007': 120491, '2015': 102657, '2014': 106849, '2008': 117375, '2009': 106018, '2011': 107203, '2010': 105639, '2013': 111286, '2012': 111798})

In [25]:
crimesFinal.filter(lambda x : x.Offense=='BURGLARY')\
            .map(lambda x:x.Occurrence_Year)\
            .countByValue()

defaultdict(<type 'int'>, {'2006': 23069, '2007': 21715, '2015': 14967, '2014': 16832, '2008': 20732, '2009': 19441, '2011': 18860, '2010': 18700, '2013': 17419, '2012': 19309})

In [26]:
crimesFinal.persist()

PythonRDD[15] at RDD at PythonRDD.scala:48

In [27]:
import gmplot
gmap = gmplot.GoogleMapPlotter(37.428, -122.145, 16).from_geocode("New York City")



In [28]:
b_lats = crimesFinal.filter(lambda x : x.Offense=='BURGLARY' and x.Occurrence_Year=="2015")\
                    .map(lambda x:extractCoords(x.Location_1)[0])\
                    .collect()

In [29]:
b_lons = crimesFinal.filter(lambda x : x.Offense=='BURGLARY' and x.Occurrence_Year=="2015")\
                    .map(lambda x:extractCoords(x.Location_1)[1])\
                    .collect()

In [30]:
gmap.scatter(b_lats, b_lons, '#DE1515', size=40, marker=False)

In [54]:
#help(gmap.draw)
gmap.draw("/home/test/mymap.html")

In [32]:
import os
os.getcwd() # get the current working directory

'/mnt/resource/hadoop/yarn/local/usercache/livy/appcache/application_1515736974360_0004/container_1515736974360_0004_01_000001'

In [45]:
os.listdir(os.getcwd()) # get the list of dirs in the current dir

['sqoop', 'ambari-qa', 'spark', 'zookeeper', 'falcon', 'storm', 'hdfs', 'yarn', 'sshuser', 'hive', 'hbase', 'ranger', 'solr', 'oozie', 'kafka', 'mapred', 'kms', 'ams', 'tez', 'hcat', 'livy']

In [37]:
 print(os.path.expanduser('~')) # expand the path

/home

In [52]:
os.chdir('/home/test') # change the dir

In [40]:
import getpass # get the username 
getpass.getuser()

'livy'

In [46]:
import socket # get hostname
socket.gethostname()

'wn3-dexter'

Had to ssh to the cluster and use SCP to copy the file to the head node in the HDInsight cluster and then perform a SCP from my local workstation to get it